In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Grid
N = 4096
L = 2*np.pi
x = np.linspace(0,2.*np.pi,N,endpoint=False)
alpha = 2.*np.pi/L

kx = np.fft.fftfreq(N)* N * alpha

dt = 1.e-7
nu = 1.0

In [ ]:
def conv(uhat,alias='No'):
    N = uhat.size
    what = np.zeros_like(uhat)
    # no dealiasing
    if alias == 'No':
        kx = np.fft.fftfreq(N)*N
        w = np.fft.ifft(uhat)*np.fft.ifft(1.j*kx*uhat)
        what = np.fft.fft(w)
    
    if alias == '3/2':
        M = int(3*N/2)
        kx = np.fft.fftfreq(M)*M
        uhatn = np.zeros(M,dtype=np.complex)
        uhatn[:N//2] = uhat[:N//2]
        uhatn[-1:-N//2:-1] = uhat[-1:-N//2:-1]
        w = np.fft.ifft(uhatn)*np.fft.ifft(1.j*kx*uhatn)
        whatn = np.fft.fft(w)
        what[:N//2] = 1.5*whatn[:N//2]
        what[-1:-N//2:-1] = 1.5*whatn[-1:-N//2:-1]
        
    if alias == '2/3':
        kx = np.fft.fftfreq(N)*N
        # filter the uhat
        Filter = np.ones_like(kx)
        Filter[np.abs(kx)>=2.*np.max(kx)/3.] = 0.0
        w = np.fft.ifft(uhat*Filter)*np.fft.ifft(1.j*kx*uhat*Filter)
        what = np.fft.fft(w)
    return what

In [ ]:
nt = int(0.001/dt)

In [ ]:
A = 25; B = 16;
u0 = 3*A**2/np.cosh(0.5*(A*(x-np.pi+2.)))**2 + 3*B**2/np.cosh(0.5*(B*(x-np.pi+1)))**2
uhat = np.fft.fft(u0)

A1 = 1./(1.+nu*dt*kx**2)
A2 = A1*dt

for nt in range(nt):
    time = (nt+1)*dt
    
    what = conv(uhat,alias='3/2')
    uhat = A1*uhat - A2*what
    
u3_2 = np.real(np.fft.ifft(uhat))

In [ ]:
A = 25; B = 16;
u0 = 3*A**2/np.cosh(0.5*(A*(x-np.pi+2.)))**2 + 3*B**2/np.cosh(0.5*(B*(x-np.pi+1)))**2
uhat = np.fft.fft(u0)

A1 = 1./(1.+nu*dt*kx**2)
A2 = A1*dt

for nt in range(nt):
    time = (nt+1)*dt
    
    what = conv(uhat,alias='2/3')
    uhat = A1*uhat - A2*what
    
u2_3 = np.real(np.fft.ifft(uhat))

In [ ]:
A = 25; B = 16;
u0 = 3*A**2/np.cosh(0.5*(A*(x-np.pi+2.)))**2 + 3*B**2/np.cosh(0.5*(B*(x-np.pi+1)))**2
uhat = np.fft.fft(u0)
A1 = 1./(1.+nu*dt*kx**2)
A2 = A1*dt

for nt in range(nt):
    time = (nt+1)*dt
    
    what = conv(uhat,alias='No')
    uhat = A1*uhat - A2*what
    
uNone = np.real(np.fft.ifft(uhat))

In [ ]:
plt.figure()
plt.plot(x,u0,label='U0')
plt.plot(x,u2_3,label='2/3 rule')
plt.plot(x,u3_2,label='3/2 rule')
plt.plot(x,uNone,label='None')
plt.legend()